In [ ]:
# Import required libraries
import os
import subprocess
import shutil

In [ ]:
# Define paths
base_dir = "/mnt/d/school/wildfire/hysplit/organized/hysplit/HYSPLIT"
working_dir = os.path.join(base_dir, "working")
exec_dir = os.path.join(base_dir, "exec")
bdy_files_dir = os.path.join(base_dir, "bdyfiles")
graphics_dir = os.path.join(base_dir, "graphics")
output_dir = "/mnt/d/school/wildfire/hysplit/organized/data/conc_output"

# Meteorological data file
met_file = "gdas1.jan25.w2"  # Adjust to your actual file

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")

In [ ]:
def create_control_file(output_dir, met_dir, met_file):
    """Create a HYSPLIT CONTROL file for PM2.5 concentration simulation"""
    control_path = os.path.join(met_dir, "CONTROL")
    
    # Make sure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(control_path, 'w') as f:
        f.write("25 01 10 00\n")             # Start year, month, day, hour
        f.write("1\n")                       # Number of starting locations
        f.write("34.03 -118.33 500\n")       # Lat, lon, height (m)
        f.write("72\n")                      # Run duration (hours)
        f.write("0\n")                       # Vertical motion option
        f.write("10000.0\n")                 # Mass release amount
        f.write("1\n")                       # Number of met files
        f.write(f"{met_dir}/\n")             # Met file directory
        f.write(f"{met_file}\n")             # Met file name
        f.write("1\n")                       # Pollutant type count
        f.write("PM25\n")                    # Pollutant name
        f.write("1.0\n")                     # Emission rate (per hour)
        f.write("1.0\n")                    # Averaging time (hours)
        f.write("00 00 00 00 00\n")          # Release start (yy mm dd hh mm)
        f.write("1\n")                       # Number of grid levels
        f.write("0.0 0.0\n")                 # Grid center (lat, lon)
        f.write("0.05 0.05\n")               # Grid spacing (lat, lon)
        f.write("40.0 40.0\n")               # Grid span (lat, lon)
        f.write("./\n")                      # Output directory
        f.write("cdump\n")                   # Output file name
        f.write("1\n")                       # Number of vertical levels
        f.write("100\n")                     # Height of level (m)
        f.write("00 00 00 00 00\n")          # Sampling start (yy mm dd hh mm)
        f.write("00 03 00 00 00\n")          # Sampling stop (yy mm dd hh mm)
        f.write("00 01 00\n")                # Sampling interval (hh mm ss)
        f.write("1\n")                       # Number of parameters in addition
        f.write("0.0 0.0 0.0\n")             # Particle parameters
        f.write("0.0 0.0 0.0 0.0 0.0\n")     # Deposition parameters
        f.write("0.0 0.0 0.0\n")             # Additional deposition parameters
        f.write("0.0\n")                     # Radioactive decay half-life (days)
        f.write("0.0\n")                     # Pollutant resuspension factor (1/m)
    
    return control_path

In [ ]:
def create_ascdata_cfg(working_dir, bdy_files_dir):
    """Create ASCDATA.CFG file for land use data"""
    ascdata_path = os.path.join(working_dir, "ASCDATA.CFG")
    
    with open(ascdata_path, 'w') as f:
        f.write("-90.0   -180.0  lat/lon of lower left corner\n")
        f.write("1.0     1.0     lat/lon spacing in degrees\n")
        f.write("180     360     lat/lon number of data points\n")
        f.write("2               default land use category\n")
        f.write("0.2             default roughness length (m)\n")
        f.write(f"{bdy_files_dir}/  directory of files\n")
    
    return ascdata_path

In [ ]:
def run_hysplit_conc(exec_path, working_dir, output_dir):
    """Run the HYSPLIT concentration model"""
    # Change to working directory
    original_dir = os.getcwd()
    os.chdir(working_dir)
    
    # Remove previous output files if they exist
    if os.path.exists("cdump"):
        os.remove("cdump")
    if os.path.exists("SETUP.CFG"):
        os.remove("SETUP.CFG")
    
    # Run HYSPLIT
    try:
        subprocess.run([exec_path], check=True)
        print("HYSPLIT concentration run completed successfully")
        
        # Copy output files to output directory
        if os.path.exists("cdump"):
            shutil.copy("cdump", os.path.join(output_dir, "cdump"))
            print(f"Copied cdump to {output_dir}")
        else:
            print("Warning: cdump file not created")
        
        # Return to original directory
        os.chdir(original_dir)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running HYSPLIT: {e}")
        # Return to original directory
        os.chdir(original_dir)
        return False

In [ ]:
# Create configuration files
create_ascdata_cfg(working_dir, bdy_files_dir)
control_path = create_control_file(output_dir, working_dir, met_file)
print(f"Created CONTROL file at: {control_path}")

In [ ]:
# Paths to executables
hycs_std_path = os.path.join(exec_dir, "hycs_std")
concplot_path = os.path.join(exec_dir, "concplot")

# Run HYSPLIT concentration model
if run_hysplit_conc(hycs_std_path, working_dir, output_dir):
    print(f"Finished! PM2.5 concentration output files are in {output_dir}")
else:
    print("HYSPLIT run failed")

In [ ]:
# Cell to run the con2asc binary from data/conc_output directory
!cd data/conc_output && ./con2asc -icdump -ocdump_pm25 cdump_010_01

In [ ]:
# Prepare PM2.5 concentration data for ConvLSTM training
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import pickle

# Directory containing cdump files
data_dir = "data/conc_output"  # Update this to your actual directory
output_dir = "convlstm_data"   # Directory for processed data

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to parse cdump file
def parse_cdump(file_path):
    """Parse HYSPLIT cdump file into a pandas DataFrame"""
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Find the data section (after the header)
    for i, line in enumerate(lines):
        if 'DAY HR' in line:
            header_line = i
            break
    
    # Column names from the header line
    columns = lines[header_line].strip().split()
    
    # Parse data lines
    data = []
    for line in lines[header_line+1:]:
        if line.strip():  # Skip empty lines
            values = line.strip().split()
            if len(values) == len(columns):
                data.append(values)
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    # Convert data types
    for col in df.columns:
        if col in ['DAY', 'HR']:
            df[col] = df[col].astype(int)
        elif col in ['LAT', 'LON']:
            df[col] = df[col].astype(float)
        else:  # PM2500100 or other concentration columns
            # Handle scientific notation with 'E' format
            df[col] = df[col].apply(lambda x: float(x.replace('E', 'e')))
    
    return df

# Find all cdump files
cdump_files = sorted(glob.glob(os.path.join(data_dir, "cdump_*_*")))
if not cdump_files:
    # Try the current directory if data_dir doesn't work
    cdump_files = sorted(glob.glob("cdump_*_*"))

if not cdump_files:
    print("No cdump files found. Please check the file path.")
else:
    print(f"Found {len(cdump_files)} cdump files")
    
    # Set map boundaries (matches your simulation area)
    lat_min, lat_max = 33.6, 34.3
    lon_min, lon_max = -118.6, -117.9
    
    # Create grid for interpolation - 40x40 as requested
    grid_resolution = 40
    grid_lon = np.linspace(lon_min, lon_max, grid_resolution)
    grid_lat = np.linspace(lat_min, lat_max, grid_resolution)
    grid_lon_mesh, grid_lat_mesh = np.meshgrid(grid_lon, grid_lat)
    
    # Container for sequence data
    sequence_data = []
    sequence_info = []  # To store metadata (day, hour)
    
    # Process each cdump file
    for cdump_file in cdump_files:
        # Extract time information from filename
        filename = os.path.basename(cdump_file)
        day_hour = filename.replace('cdump_', '').split('_')
        day = int(day_hour[0])
        hour = int(day_hour[1])
        
        # Parse the cdump file
        df = parse_cdump(cdump_file)
        
        if len(df) > 3:  # Need at least 3 points for interpolation
            # Extract coordinates and concentration values
            points = df[['LON', 'LAT']].values
            values = df.iloc[:, -1].values  # Assuming the last column is concentration
            
            # Interpolate concentration values onto the standardized grid
            # Using linear interpolation instead of cubic
            grid_conc = griddata(points, values, (grid_lon_mesh, grid_lat_mesh), 
                               method='linear', fill_value=0)
            
            # Save the grid as a numpy array for ConvLSTM training
            sequence_data.append(grid_conc)
            sequence_info.append({'day': day, 'hour': hour})
            
            # Also save as visualization for inspection
            fig, ax = plt.figure(figsize=(8, 8)), plt.subplot()
            im = ax.imshow(grid_conc, origin='lower', cmap='viridis')
            plt.colorbar(im, ax=ax, label='PM2.5 Concentration')
            plt.title(f'PM2.5 Grid for ConvLSTM - Day {day}, Hour {hour}')
            plt.savefig(os.path.join(output_dir, f'pm25_grid_day{day}_hour{hour}.png'), 
                       dpi=150, bbox_inches='tight')
            plt.close(fig)
            
            print(f"Processed Day {day}, Hour {hour}")
        else:
            print(f"Not enough data points in {filename} for interpolation")
    
    # Convert sequence data to numpy array
    sequence_data = np.array(sequence_data)
    
    # Save as pickle for easy loading in training
    with open(os.path.join(output_dir, 'pm25_sequence_data.pkl'), 'wb') as f:
        pickle.dump({
            'data': sequence_data,
            'metadata': sequence_info,
            'grid_info': {
                'lat_min': lat_min,
                'lat_max': lat_max,
                'lon_min': lon_min,
                'lon_max': lon_max,
                'resolution': grid_resolution
            }
        }, f)
    
    # Also save as numpy array
    np.save(os.path.join(output_dir, 'pm25_sequence_data.npy'), sequence_data)
    
    print(f"Saved sequence data with shape: {sequence_data.shape}")
    print(f"All processing complete. Data saved to {output_dir}")

In [ ]:
def sliding_window_of(frames, sample_size, rows, cols, channels):
    n_samples = len(frames) - sample_size
    samples = np.empty((n_samples, sample_size, rows, cols, channels))
    for i in range(n_samples):
        samples[i] = np.array([frames[j] for j in range(i, i + sample_size)])
        
    return samples

In [ ]:
import json
import requests


def get_airnow_data(
    start_date, end_date, 
    lon_bottom, lat_bottom, lon_top, lat_top,
    airnow_api_key=None
):
    # get airnow data from the EPA
    if os.path.exists('data/airnow.json'):
        print("data/airnow.json already exists; skipping request...")
    else:
        # preprocess a few parameters
        date_start = pd.to_datetime(start_date).isoformat()[:13]
        date_end = pd.to_datetime(end_date).isoformat()[:13]
        bbox = f'{lon_bottom},{lat_bottom},{lon_top},{lat_top}'
        URL = "https://www.airnowapi.org/aq/data"
                
        # defining a params dict for the parameters to be sent to the API
        PARAMS = {
            'startDate':date_start,
            'endDate':date_end,
            'parameters':'PM25',
            'BBOX':bbox,
            'dataType':'B',
            'format':'application/json',
            'verbose':'0',
            'monitorType':'2',
            'includerawconcentrations':'1',
            'API_KEY':airnow_api_key
        }
        
        # sending get request and saving the response as response object
        response = requests.get(url = URL, params = PARAMS)
    
        # extracting data in json format, then download
        airnow_data = response.json()
        with open('data/airnow.json', 'w') as file:
            json.dump(airnow_data, file)
            print("JSON data saved to data/airnow.json")
        
    # open json file and convert to dataframe
    with open('data/airnow.json', 'r') as file:
        airnow_data = json.load(file)
    airnow_df = pd.json_normalize(airnow_data)

    # group station data by time
    list_df = [group for name, group in airnow_df.groupby('UTC')]
    return list_df

In [ ]:
def preprocess_ground_sites(df, dim, latMax, latMin, lonMax, lonMin):
    latDist, lonDist = abs(latMax - latMin), abs(lonMax - lonMin)
    unInter = np.zeros((dim,dim))
    dfArr = np.array(df[['Latitude','Longitude','Value']])
    for i in range(dfArr.shape[0]):
        # Calculate x
        x = int(((latMax - dfArr[i,0]) / latDist) * dim)
        if x >= dim:
            x = dim - 1
        if x <= 0:
            x = 0
        # Calculate y
        y = dim - int(((lonMax + abs(dfArr[i,1])) / lonDist) * dim)
        if y >= dim:
            y = dim - 1
        if y <= 0:
            y = 0
        if dfArr[i,2] < 0:
            unInter[x,y] = 0
        else:
            unInter[x,y] = dfArr[i,2]
    return unInter

In [ ]:
def interpolate_frame(f, dim):
    i = 0
    interpolated = []
    count = 0
    idx = 0
    x_list = []
    y_list = []
    values = []
    for x in range(f.shape[0]):
        for y in range(f.shape[1]):
            if f[x,y] != 0:
                x_list.append(x)
                y_list.append(y)
                values.append(f[x,y])
    coords = list(zip(x_list,y_list))
    try:
        interp = NearestNDInterpolator(coords, values)
        X = np.arange(0,dim)
        Y = np.arange(0,dim)
        X, Y = np.meshgrid(X, Y)
        Z = interp(X, Y)
    except ValueError:
        Z = np.zeros((dim,dim))
    interpolated = Z
    count += 1
    i += 1
    interpolated = np.array(interpolated)
    return interpolated

In [ ]:
# we scale each sample relative to other samples
def std_scale(data):
    mean = np.mean(data)
    stddev = np.std(data)
    return (data - mean) / stddev

In [ ]:
# get data, process ground sites, interpolate, create samples
list_df = get_airnow_data(
    start_date, end_date,
    lon_bottom, lat_bottom, lon_top, lat_top
)
list_unInter = [preprocess_ground_sites(df, dim, lat_top, lat_bottom, lon_top, lon_bottom) for df in list_df]
list_inter = [interpolate_frame(unInter, dim) for unInter in list_unInter]
frames = np.expand_dims(np.array(list_inter), axis=-1)
X_airnow = sliding_window_of(frames, frames_per_sample, dim, dim, 1)

print(X_airnow.shape)